In [13]:
%load_ext autoreload
%autoreload 2
%run ./initializer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Path (/Users/rik/Documents/VU/DMT/DataMiningTechniquesA1) already exists in sys.path


In [14]:
import polars as pl
from datetime import datetime, date
import pandas as pd
from matplotlib import pyplot as plt
from data_loading import DataLoader

In [15]:
data_loader = DataLoader()
df = data_loader.load_and_transform_data("12h")